In [2]:
from gudiff_model import PDBDataSet_GraphCon
from gudiff_model.Graph_UNet import GraphUNet
from data_rigid_diffuser.diffuser import FrameDiffNoise

from se3_transformer.model.fiber import Fiber
import torch
import os
import logging
from datetime import datetime
from collections import defaultdict
import time
import tree
from se3_transformer.model.FAPE_Loss import FAPE_loss, Qs2Rs, normQ
from torch import einsum
import numpy as np
import se3_diffuse.utils as du
import util.framediff_utils as fu
from data_rigid_diffuser import rigid_utils as ru
import copy
import util.pdb_writer 
from experiment.Experiment import Experiment
import json
import argparse
import pandas as pd


/home/nwoodall/miniconda3/envs/gud/lib/python3.9/site-packages/torchdata/datapipes/__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()
/home/nwoodall/miniconda3/envs/gud/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
with open('configs/gun_cluster_nf.json', 'r') as f:
            conf = json.load(f)
exp = Experiment(conf, name='t2', ckpt_model=None, ckpt_opt=None,swap_metadir=True)

In [5]:
#exp.dataset.csv[exp.dataset.csv['cluster']==0]
(train_loader, valid_loader) = exp.create_dataset()
df = exp.dataset.csv

In [ ]:
epoch_log = self.train_epoch(train_loader, valid_loader, epoch=epoch, return_logs=return_logs)

In [6]:
for train_feats in train_loader:
    batch_feats= tree.map_structure(
                    lambda x: x.to('cuda'), train_feats)
    noised_dict =   {'CA': batch_feats['CA_noised'] ,
                        'N_CA': batch_feats['N_CA_noised'].unsqueeze(-2) ,
                        'C_CA': batch_feats['C_CA_noised'].unsqueeze(-2)  }
    
    break

2iaf 140 cluster:  80549
4g08 140 cluster:  48942
4g08 140 cluster:  48942
1ej8 140 cluster:  201241


In [8]:
batch_feats.keys()

dict_keys(['chain_idx', 'residue_index', 'res_mask', 'CA', 'N_CA', 'C_CA', 'file_path_index', 'CA_noised', 'N_CA_noised', 'C_CA_noised', 't', 'score_scale'])

In [11]:
batch_feats['chain_idx'].shape

torch.Size([4, 140])

In [12]:
L = batch_feats['CA'].shape[1]
B = batch_feats['CA'].shape[0]
CA_t  = batch_feats['CA']
NC_t = CA_t +  batch_feats['N_CA']
CC_t = CA_t +  batch_feats['C_CA']
true =  torch.cat((NC_t,CA_t,CC_t),dim=2).reshape(B,L,3,3)

CA_n  = batch_feats['CA_noised'].reshape(B, L, 3)
NC_n = CA_n + batch_feats['N_CA_noised'].reshape(B, L, 3)
CC_n = CA_n + batch_feats['C_CA_noised'].reshape(B, L, 3)
noise_xyz =  torch.cat((NC_n,CA_n,CC_n),dim=2).reshape(B,L,3,3)

x = exp._graphmaker.prep_for_network(noised_dict, cuda= exp.use_cuda)

In [19]:
b_graph, nf, ef, b_graph_mp, nf_mp, ef_mp, b_graph_mps, nf_mps, ef_mps, b_graph_mpRev = x

In [29]:
nf_mp['0'].shape

torch.Size([700, 32, 1])

In [32]:
ca_nodes = int(b_graph.batch_num_nodes()[0])
mp_nodes = int(b_graph_mp.batch_num_nodes()[0]) #ca+mp nodes number

#SE3 Attention Transformer, c_alpha
embed_t = exp._model.embed_t( batch_feats['t'])
t_nf = exp._model.concat_t(nf, embed_t, use_deg1=False) #concat_t on
nf_ca_down_out = exp._model.down_ca(b_graph, t_nf, ef)  


/home/nwoodall/miniconda3/envs/gud/lib/python3.9/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/nwoodall/miniconda3/envs/gud/lib/python3.9/site-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [34]:
nf_ca_down_out['0'].shape

torch.Size([560, 64, 1])

In [36]:
nf_mp['0'].shape

torch.Size([700, 32, 1])

In [ ]:
def concat_mp_feats(self, ca_feats_in, mp_feats):

    nf0_c = ca_feats_in['0'].shape[-2]
    nf1_c = ca_feats_in['1'].shape[-2]

    out0_cat_shape = (self.B,self.ca_nodes,-1,1)
    mp0_cat_shape  = (self.B,self.mp_nodes,-1,1)
    out1_cat_shape = (self.B,self.ca_nodes,-1,3)
    mp1_cat_shape  = (self.B,self.mp_nodes,-1,3)

    nf_c = {} #nf_cat
    nf_c['0'] = torch.cat((ca_feats_in['0'].reshape(out0_cat_shape), 
                                mp_feats['0'].reshape(mp0_cat_shape)[:,-(self.mp_nodes-self.ca_nodes):,:,:]),
                            axis=1).reshape((-1,nf0_c,1))

    nf_c['1'] = torch.cat((ca_feats_in['1'].reshape(out1_cat_shape), 
                                mp_feats['1'].reshape(mp1_cat_shape)[:,-(self.mp_nodes-self.ca_nodes):,:,:]),
                            axis=1).reshape((-1,nf1_c,3))

    return nf_c

In [18]:
out = exp._model(x, batch_feats['t'])

/home/nwoodall/miniconda3/envs/gud/lib/python3.9/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 64 but got size 32 for tensor number 1 in the list.

In [27]:
def make_pe_encoding(n_nodes=65, embed_dim = 12, scale = 40, cast_type=torch.float32, print_out=False):
    #positional encoding of node, scale dimension optimized for 65
    i_array = np.arange(1,(embed_dim/2)+1)
    wk = (1/(scale**(i_array*2/embed_dim)))
    t_array = np.arange(n_nodes)
    si = torch.tensor(np.sin(wk*t_array.reshape((-1,1))))
    ci = torch.tensor(np.cos(wk*t_array.reshape((-1,1))))
    pe = torch.stack((si,ci),axis=2).reshape(t_array.shape[0],embed_dim).type(cast_type)
    
    if print_out == True:
        for x in range(int(n_nodes/100)):
            print(np.round(pe[x],1))
    
    return pe

In [ ]:
pe = make_pe_encoding(n_nodes=512, embed_dim = 12, scale = 40, cast_type=torch.float32, print_out=False)

tensor([0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.])
tensor([0.5000, 0.9000, 0.3000, 1.0000, 0.2000, 1.0000, 0.1000, 1.0000, 0.0000,
        1.0000, 0.0000, 1.0000])
tensor([0.9000, 0.5000, 0.6000, 0.8000, 0.3000, 1.0000, 0.2000, 1.0000, 0.1000,
        1.0000, 0.0000, 1.0000])
tensor([ 1.0000, -0.1000,  0.8000,  0.6000,  0.5000,  0.9000,  0.3000,  1.0000,
         0.1000,  1.0000,  0.1000,  1.0000])
tensor([ 0.8000, -0.6000,  0.9000,  0.4000,  0.6000,  0.8000,  0.3000,  0.9000,
         0.2000,  1.0000,  0.1000,  1.0000])


In [26]:
pe.shape

torch.Size([512, 12])